In [ ]:
!pip install opencv-python numpy matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import random

input_dir = 'exercise'
output_dir = 'output_random_transform'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def inverse_image(img):
    return 255 - img

def gamma_correction(img, gamma=2.2):
    norm = img / 255.0
    corrected = np.power(norm, gamma)
    return np.uint8(corrected * 255)

def log_transform(img):
    img_float = img.astype(np.float32)
    log_image = np.log1p(img_float)
    log_image = cv2.normalize(log_image, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(log_image)

def histogram_equalization(img):
    if len(img.shape) == 2:
        return cv2.equalizeHist(img)
    else:
        ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

def contrast_stretching(img):
    in_min = np.min(img)
    in_max = np.max(img)
    stretched = (img - in_min) * (255 / (in_max - in_min))
    return np.uint8(stretched)

def random_rgb_order(img):
    channels = cv2.split(img)
    random.shuffle(channels)
    return cv2.merge(channels)

In [ ]:
transformations = {
    'I': ('Inverse', inverse_image),
    'G': ('Gamma Correction', gamma_correction),
    'L': ('Log Transform', log_transform),
    'H': ('Histogram Equalization', histogram_equalization),
    'C': ('Contrast Stretching', contrast_stretching)
}

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
        path = os.path.join(input_dir, filename)
        img = cv2.imread(path)

        rgb_img = random_rgb_order(img)

        code = random.choice(list(transformations.keys()))
        name, transform_fn = transformations[code]
        print(f"Đang xử lý {filename} với phép biến đổi: {name}")

        result = transform_fn(rgb_img)

        out_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_{code}.png")
        cv2.imwrite(out_path, result)

        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB))
        plt.title("RGB Shuffled")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        cmap = 'gray' if len(result.shape) == 2 else None
        plt.imshow(result, cmap=cmap)
        plt.title(name)
        plt.axis("off")
        plt.show()

print(f"✅ Ảnh đã được xử lý và lưu vào thư mục '{output_dir}'")